# Saving multiple netcdf dataset into zarr in the google bucket using the cluster 
Trial and errors and more info on some of the steps can be found in nc_to_zarr_testing

In [1]:
import dask
import dask.array as da
import dask.delayed
from dask.distributed import Client
import dask_gateway
import numpy as np
import xarray as xr
xr.set_options(display_style="html")
import fsspec
import gcsfs
import json

### 1. Start a cluster

In [40]:
# get the dask-gateway version
dask_gateway.__version__
# show the default dask-gateway settings
dask.config.config['gateway']
#default gateway call
gateway = dask_gateway.Gateway()
# default new_cluster call
cluster = gateway.new_cluster()
#gateway = Gateway()
gateway.list_clusters()
# the dashboard_link property will show the link that can be pasted into the Dask labextension
cluster.dashboard_link

distributed.client - ERROR - Failed to reconnect to scheduler after 10.00 seconds, closing client
_GatheringFuture exception was never retrieved
future: <_GatheringFuture finished exception=CancelledError()>
asyncio.exceptions.CancelledError
distributed.client - ERROR - Failed to reconnect to scheduler after 10.00 seconds, closing client
distributed.client - ERROR - Failed to reconnect to scheduler after 10.00 seconds, closing client
_GatheringFuture exception was never retrieved
future: <_GatheringFuture finished exception=CancelledError()>
asyncio.exceptions.CancelledError
_GatheringFuture exception was never retrieved
future: <_GatheringFuture finished exception=CancelledError()>
asyncio.exceptions.CancelledError


'/services/dask-gateway/clusters/prod.5e434112b31d49a39faae31121a50e29/status'

In [41]:
# scale cluster to 8 workers using the scale() method
cluster.scale(8)
# connect a client
# the distributed client is used for running parallel tasks with Dask
client = Client(cluster)
cluster

### 2. load the netcdf
(defining chunk size to ensure itis a dask array)

In [42]:
# the four line below come from https://github.com/pangeo-data/pangeo-datastore/issues/116#issuecomment-729928936
gcs = gcsfs.GCSFileSystem()
url = 'gs://ldeo-glaciology/bedmachine/BedMachineAntarctica_2019-11-05_v01.nc'  
openfile = gcs.open(url, mode='rb') 
bm = xr.open_dataset(openfile, chunks=3000)

# remove the variable mapping because it was causing an error in the write to zarr
bm.attrs = {**bm.attrs, **bm.mapping.attrs} # but keep the information in the attributes of the whole dataset. 
bm = bm.drop('mapping')   # remove the variable. 


### 3. Add the token to the gcs object to make sure all the workers can write to the bucket)

In [44]:
with open('../secrets/ldeo-glaciology-bc97b12df06b.json') as token_file:
    token = json.load(token_file)
gcs = gcsfs.GCSFileSystem(token=token)

### 4. Save to zarr


In [47]:
mapper = gcs.get_mapper('gs://ldeo-glaciology/bedmachine/bm6.zarr')
bm.to_zarr(mapper, mode='w');

### 5. Reload the zarr and check that it is the same as the original

In [ ]:
bm_reloaded = xr.open_zarr(mapper)  
bm_reloaded.identical(bm)

In [38]:
cluster.shutdown()